# YOLO Multi-Ingredient Detection — Cleaned Version

This notebook is a **cleaned-up version** of my YOLO training workflow.  

Originally, multiple experiments were performed using various datasets, methods, and hyperparameter configurations. For clarity and readability, all trial-and-error code, intermediate outputs, and unnecessary steps have been removed.  

This notebook contains only the **essential code** needed to reproduce the YOLO training for multi-ingredient detection in the
project.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install fiftyone
!pip install torch torchvision
!pip install ultralytics

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import os
import random
import yaml
from shutil import copy2

# Define target classes and maximum samples per class
classes = ["Apple","Banana", "Grape","Milk","Bell pepper", "Bread", "Broccoli", "Cucumber", "Egg (Food)", "Zucchini", "Lemon", "Orange", "Tomato", "Cabbage", "Carrot"]
sample_limit_per_class = 500

# Load Open Images v7 dataset with specified classes and sample limit
view = foz.load_zoo_dataset(
    "open-images-v7",
    split="train",
    label_types=["detections"],
    classes=classes,
    max_samples=sample_limit_per_class * len(classes),
)

# Print schema to verify label fields
print(view.schema)

# Field containing object detection annotations
label_field = "ground_truth"

# Set dataset export directories
export_dir = "/content/open_images_data"
train_dir = os.path.join(export_dir, 'train')
val_dir = os.path.join(export_dir, 'val')
test_dir = os.path.join(export_dir, 'test')

# Create directories for train, validation, and test splits
for subset in [train_dir, val_dir, test_dir]:
    os.makedirs(os.path.join(subset, 'images'), exist_ok=True)
    os.makedirs(os.path.join(subset, 'labels'), exist_ok=True)

# Shuffle and split dataset into train (80%), validation (10%), and test (10%)
samples = list(view)
random.shuffle(samples)
train_samples = samples[:int(0.8 * len(samples))]
val_samples = samples[int(0.8 * len(samples)):int(0.9 * len(samples))]
test_samples = samples[int(0.9 * len(samples)):]

# Function to copy images and generate YOLO-format label files
def copy_files(samples, subset_dir):
    for sample in samples:
        # Copy image file to target directory
        image_file = sample.filepath
        image_filename = os.path.basename(image_file)
        copy2(image_file, os.path.join(subset_dir, 'images', image_filename))

        # Ensure sample contains valid detections
        if label_field in sample and sample[label_field] is not None and len(sample[label_field].detections) > 0:
            # Write YOLO-formatted labels
            label_filename = os.path.splitext(image_filename)[0] + '.txt'
            label_path = os.path.join(subset_dir, 'labels', label_filename)

            with open(label_path, 'w') as label_file:
                for detection in sample[label_field].detections:
                    # Only include detections for target classes
                    if detection.label in classes:
                        class_id = classes.index(detection.label)
                        bbox = detection.bounding_box
                        x_center = max(0, min(1, bbox[0] + bbox[2] / 2))
                        y_center = max(0, min(1, bbox[1] / 2))
                        width = max(0, min(1, bbox[2]))
                        height = max(0, min(1, bbox[3]))

                        # Write normalized bounding box coordinates
                        label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
        else:
            print(f"Warning: No labels found for image: {image_filename}")

# Copy files and generate labels for each split
copy_files(train_samples, train_dir)
copy_files(val_samples, val_dir)
copy_files(test_samples, test_dir)

# Generate YOLO data configuration (data.yaml)
data_yaml = {
    'train': os.path.join(export_dir, 'train/images'),
    'val': os.path.join(export_dir, 'val/images'),
    'test': os.path.join(export_dir, 'test/images'),
    'nc': len(classes),  # Number of classes
    'names': classes,    # Class names
}

# Save data.yaml to export directory
yaml_path = os.path.join(export_dir, 'data.yaml')
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(data_yaml, yaml_file)

print(f"Dataset exported to: {export_dir}")
print(f"data.yaml file created at: {yaml_path}")


In [ ]:
from ultralytics import YOLO

# Load pretrained YOLO model
model = YOLO("yolo11n.pt")

# You can train the model with customized settings
results = model.train(
    data="/content/open_images_data/data.yaml",
    epochs=30,
    imgsz=640,                     # Set image size
    batch=16,
    project="/content/drive/MyDrive/YOLOv11_training results",  # Save to Google Drive
    name="yolo_experiment"
)


In [ ]:
from pathlib import Path

import os
from ultralytics import YOLO
from PIL import Image

# Define paths
model_path = '/content/drive/MyDrive/YOLOv11_training results/yolo_experiment/weights/best.pt'
input_folder = '/content/drive/MyDrive/fruits and vegies'    # Folder containing test images
output_folder = '/content/drive/MyDrive/vegetables(open_images) detection'       # Folder to save detection results

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
model = YOLO(model_path)

# Make sure output folder exists
Path(output_folder).mkdir(parents=True, exist_ok=True)

# Process each image in the input folder
for img_file in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_file)

    # Run inference
    results = model(img_path)

    # Save each result individually
    for i, result in enumerate(results):
        # Get the output path
        output_path = os.path.join(output_folder, f"annotated_{img_file}")

        # Save each annotated image to the output folder
        result.save(output_path)

    print(f"Processed {img_file}, results saved to {output_folder}")
